In [2]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical 
import numpy as np 
import pandas as pd

from sklearn.preprocessing import minmax_scale 
from sklearn.model_selection import train_test_split

from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation 
from tensorflow.keras.optimizers import Adam
import itertools

In [3]:
file_directory='../Preprocess/data_after/'
#opposite과 my_team에 대한 변수를 두어 그 값이 1이면 내 팀, 0이면 다른 팀 인 방식으로 구현
#테스트 데이터는 match.csv의 result attribute를 사용
#train_data에 사용되는 데이터는 player1.csv의 선수의 티어 값을 이용
#(X_train, y_train), (X_test, y_test) = mnist.load_data() 


In [4]:

#player1.csv의 파일을 읽어서 각 팀별 플레이어 티어 평균값을 추출
data=pd.read_csv("{0}player1.csv".format(file_directory))
tier = data.loc[:,['Team','Tier']]

team = data.sort_values('Team' ,ascending=True)
team = team.loc[:,'Team'].drop_duplicates()
team = team.to_list()
print(team)
AvgTier = tier.groupby('Team').mean()
AvgTier2 = np.random.rand(AvgTier.shape[0])
AvgTier2 = AvgTier2.reshape((31,1))*5
print(AvgTier2)


AvgTier = AvgTier.to_numpy()

Avg = np.zeros((31,2))

AvgTier = AvgTier.reshape(31)
AvgTier2 = AvgTier2.reshape(31)

Avg[:,0] = AvgTier
Avg[:,1] = AvgTier2


print(Avg)

X_train = Avg

y_train = np.zeros(31)


for i in range(AvgTier.shape[0]):
    if AvgTier[i]<AvgTier2[i]:
        y_train[i] = 1
    else:
        y_train[i] = 0

y_train = y_train.reshape((31,1))
print(y_train)

['CJ Intas', 'DRK', 'DWG KOE', 'Efriice Friics', 'Fridot BROUN', 'Gin.G', 'Groffon', 'Henwhe Lofi Ispurts', 'ISC Ivir', 'Ivir8 Wonnirs', 'Jon Eor Griin Wongs', 'KSV iSpurts', 'KT Rulstir', 'Kongzuni DregunX', 'Kungduu Munstir', 'Loov SENDBUX', 'Lungzha Gemong', 'MBP', 'NeJon i-mFori', 'Nungshom RidFurci', 'Oncridobli Morecli', 'RUX Togirs', 'Ribils Enerchy', 'SBINA Kurie', 'SK Tilicum T1', 'Semsang Gelexy', 'SiulHeiUni Pronci', 'T2', 'Tiem Dynemocs', 'bbq Ulovirs', 'i-mFori']
[[3.88418114]
 [4.53139017]
 [2.58220718]
 [0.80254716]
 [1.46668602]
 [2.01530436]
 [1.61764003]
 [1.49889216]
 [1.34066841]
 [0.51986191]
 [0.77310322]
 [0.62442493]
 [0.8486525 ]
 [2.56013262]
 [1.84040643]
 [0.77961429]
 [1.21455731]
 [3.66889284]
 [4.32518683]
 [2.89542032]
 [4.69436736]
 [1.347315  ]
 [4.01476965]
 [2.52564191]
 [2.23486484]
 [0.54176088]
 [3.54521517]
 [2.00924153]
 [4.08190734]
 [4.8942554 ]
 [2.73857085]]
[[2.57142857 3.88418114]
 [1.88235294 4.53139017]
 [1.94117647 2.58220718]
 [2.51282

In [5]:
#match.csv 파일을 읽어서 팀의 승패 결과를 test 데이터로 활용
data=pd.read_csv("{0}match.csv".format(file_directory))

X_test = data.loc[:,['side','team','result']]
answer = np.zeros(31)

for i in range(len(team)):
    for idx in X_test.index:
        if team[i]==X_test.loc[idx,'team']:
            answer[i] = X_test.loc[idx,'result']
            break

print(answer)

[1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0.]


In [9]:
TestTier = np.random.rand(AvgTier.shape[0])
TestTier = TestTier.reshape(AvgTier.shape[0])*5


TestTier2 = np.random.rand(AvgTier.shape[0])
TestTier2 = TestTier2.reshape(AvgTier.shape[0])*5

X_test = np.zeros((AvgTier.shape[0],2))

X_test[:,0] = TestTier
X_test[:,1] = TestTier2


for i in range(X_test.shape[0]):
    if answer[i]==0:
        if X_test[i,0] <X_test[i,1]:
            temp = X_test[i,0]
            X_test[i,0] = X_test[i,1]
            X_test[i,1] = temp
            
    elif answer[i]==1:
        if X_test[i,0]>X_test[i,1]:
            temp = X_test[i,0]
            X_test[i,0] = X_test[i,1]
            X_test[i,1] = temp

In [28]:

print(Test_X)
print(answer)
num_of_train_samples = X_train.shape[0] 
width = X_train.shape[1]    
print(width)
print(num_of_train_samples)


[[3.8226925  3.9192815 ]
 [2.01386366 0.96681429]
 [0.53721379 3.81876687]
 [4.8972895  1.0105046 ]
 [3.02524261 2.57555435]
 [0.87440524 0.76767991]
 [1.2235401  0.844486  ]
 [1.54408866 4.76217534]
 [0.16177842 3.65905522]
 [2.06538537 3.45263503]
 [0.27128279 2.1335574 ]
 [0.83258745 1.84071236]
 [2.61850638 2.16993598]
 [2.87630353 3.23622373]
 [2.17788317 2.16131657]
 [2.57105835 3.88136887]
 [3.0063953  4.03495737]
 [4.78039787 3.39177661]
 [3.40354059 3.96430545]
 [2.84856777 4.32092359]
 [3.97598138 0.30439955]
 [1.09034766 4.53757947]
 [3.40918638 0.71988455]
 [3.00050052 2.64214877]
 [4.93166919 2.88765999]
 [3.61018089 2.10213896]
 [2.9411963  2.27219863]
 [3.27576774 2.72103435]
 [0.38730179 3.55808015]
 [4.0379061  2.11059497]
 [2.60029413 0.04679561]]
[1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0.]
2
31


In [11]:
num_of_test_samples = X_test.shape[0] 
print(num_of_test_samples)

31


In [32]:
model = Sequential()

In [33]:
#input layer
model.add(Dense(256, input_shape=(31,2), kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3)) 

In [34]:
#hidden layer 1
model.add(Dense(256, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))

In [35]:
#hidden layer 2
model.add(Dense(256, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(Dropout(0.3))



In [36]:
#hidden layer 3
model.add(Dense(256, kernel_initializer='glorot_uniform', activation='relu')) 
model.add(Dropout(0.3))


In [37]:
number_of_class = 2
model.add(Dense(number_of_class, activation='softmax'))#soft_max 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
training_epochs = 15 
batch_size = 100 
model.fit(X_train, y_train, epochs=training_epochs, batch_size=batch_size)


Epoch 1/15


ValueError: in user code:

    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\k

In [31]:
evaluation = model.evaluate(X_test, answer, batch_size=batch_size) 
print('Accuracy: ' + str(evaluation[1]))


ValueError: in user code:

    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:1330 test_function  *
        return step_function(self, iterator)
    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:1320 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\kwcha\anaconda3\lib\site-packages\keras\engine\training.py:1313 run_step  **
        outputs = model.test_step(data)
    C:\Users\